In [1]:
import pandas
import glob
import re
import sys
import logging
import praw
import json
import numpy as np
from dateutil import parser
from datetime import datetime
import pytz

from collections import namedtuple

In [2]:
logger = logging.getLogger(__name__)
logger.handlers = []
logger.addHandler(logging.StreamHandler(sys.stdout))
logger.setLevel(logging.DEBUG)

# with open('creds.json', 'r') as fp:
#     credentials = json.load(fp)

credentials = {
    "CLIENT_ID": "NbT_QQK57EJZDQ",
    "CLIENT_SECRET": "H5BF0vTIdhfpc_Kb7tXdclxfnME",
    "CLIENT_AGENT": "memonavirus_data (by u/fallouttheprofessor)",
}

logger.info("Authenticating with Reddit...")
_reddit = praw.Reddit(
    client_id=credentials["CLIENT_ID"],
    client_secret=credentials["CLIENT_SECRET"],
    user_agent=credentials["CLIENT_AGENT"],
#     username=credentials["CLIENT_USERNAME"],
#     password=credentials["CLIENT_PASSWORD"]
)

logger2 = logging.getLogger('prawcore')
logger2.handlers = []
# logger2.setLevel(logging.DEBUG)
# logger2.addHandler(logging.StreamHandler(sys.stdout))

Authenticating with Reddit...


In [3]:
DATA_DIR = '../../data'

PATIENT_0 = 'u/woodendoors7'
FIRST_FILE = '{}/memes_infections.20.19.log'.format(DATA_DIR)

def get_files():
    def _dt_key(fname):
        parts = fname.split('.')
        return (int(parts[-3]), int(parts[-2]))
    files = sorted(glob.glob('{}/memes_infections*.log'.format(DATA_DIR)), key=_dt_key)
    start_index = files.index(FIRST_FILE)
    return files[start_index:] + files[:start_index]

get_file = lambda fname: pandas.read_csv(
    fname,
    sep='\t',
    header=None,
    names=['post_dt',
            'infectee_name',
            'infectee_post_id',
            'infector_name',
            'infector_post_id',
            'comment_or_submission'],
    parse_dates=['post_dt'],
    dtype={'infectee_name': str, 'infectee_post_id': str, 'infector_name': str, 'infector_post_id': str, 'comment_or_submission': str}
)
get_file(FIRST_FILE).head()

,post_dt,infectee_name,infectee_post_id,infector_name,infector_post_id,comment_or_submission
0,2020-03-20 19:59:59,u/woodendoors7,NaN,NaN,NaN,NaN


In [187]:
get_files()

['../../data/memes_infections.20.19.log',
 '../../data/memes_infections.20.20.log',
 '../../data/memes_infections.20.21.log',
 '../../data/memes_infections.20.22.log',
 '../../data/memes_infections.20.23.log',
 '../../data/memes_infections.21.0.log',
 '../../data/memes_infections.21.1.log',
 '../../data/memes_infections.21.2.log',
 '../../data/memes_infections.21.3.log',
 '../../data/memes_infections.21.4.log',
 '../../data/memes_infections.21.5.log',
 '../../data/memes_infections.21.6.log',
 '../../data/memes_infections.21.7.log',
 '../../data/memes_infections.21.8.log',
 '../../data/memes_infections.21.9.log',
 '../../data/memes_infections.21.10.log',
 '../../data/memes_infections.21.11.log',
 '../../data/memes_infections.21.12.log',
 '../../data/memes_infections.21.13.log',
 '../../data/memes_infections.21.14.log',
 '../../data/memes_infections.21.15.log',
 '../../data/memes_infections.21.16.log',
 '../../data/memes_infections.21.17.log',
 '../../data/memes_infections.21.18.log',
 '

In [9]:
# memes_infections.20.19.log - add row for patient 0 infection

# with open(FIRST_FILE, 'w') as f:
#     f.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(
#             '2020-03-20 19:59:59.0',
#             PATIENT_0,
#             '',
#             '',
#             '',
#             ''
#         ))

In [10]:
# fix ancestry

# 'post_dt',
# 'infectee_name',
# 'infectee_post_id',
# 'infector_name',
# 'infector_post_id',
# 'comment_or_submission'

InfectionLog = namedtuple(
    'InfectionLog',
    ['post_dt', 'infectee_name', 'infectee_post_id', 'infector_name', 'infector_post_id', 'comment_or_submission']
)

def get_missing_infection_logs(user, infectee, infectee_data, max_date=None):
    print("Getting missing infection logs for user '{}' from infectee '{}'".format(user, infectee))
    # i1->i2, i2->i3, i3->i4 ... i5->i6 (i2<-i11<-i5)
    possible_infections = []
    user_comments = _reddit.redditor(user.lstrip('u/')).comments.new(limit=None)
    logs = []
    try:
        for comment in user_comments:
            if comment.subreddit_id == 't5_2qjpg':
                comment_dt = datetime.utcfromtimestamp(comment.created_utc)
                comment_dt = comment_dt.replace(tzinfo=pytz.utc).astimezone(tz=pytz.timezone('America/New_York'))
                if max_date and comment_dt > max_date:
                    # ensure comment date is between infector date and first infectee date
                    continue
                parent = comment.parent()
                parent_author_name = parent.author.name if parent and parent.author else None
                comment_author_name = comment.author.name if comment and comment.author else None
                if not parent_author_name or comment_author_name is None or parent_author_name == comment_author_name:
                    # deleted users are ignored, just not sure how this would be handled
                    continue
                parent_author_name = 'u/{}'.format(parent_author_name)
                comment_author_name = 'u/{}'.format(comment_author_name)
                if parent_author_name not in infectee_data and parent.author_flair_text and "INFECTED" in parent.author_flair_text:
                    try:
                        infectee_data[parent_author_name] = {}
                        logs = get_missing_infection_logs(parent_author_name, user, infectee_data, max_date=None)
                    except Exception as e:
                        print("Recursive error: {}".format(e))
                if parent_author_name in infectee_data and infectee_data[parent_author_name] and comment_dt > infectee_data[parent_author_name]['post_dt']:
                    possible_infections.append(
                        InfectionLog(
                            str(comment_dt),
                            comment_author_name,
                            comment.id,
                            parent_author_name,
                            parent.id,
                            "C" if type(parent) is praw.models.Comment else "S"
                        )
                    )
    except Exception as e:
        raise Exception("Failed to get comments for user '{}': {}".format(user, e)) from e
    if not possible_infections:
        # could not find missing link
        raise Exception("Failed to find missing links for infector '{}'".format(user))
    # best guess is that it's the infector with the earliest infection date
    best_likely_infection = next(iter(sorted(possible_infections, key=lambda x: parser.parse(x.post_dt))))
    infectee_data[user] = {'post_dt': parser.parse(best_likely_infection.post_dt).replace(tzinfo=pytz.timezone('America/New_York')), 'infector_name': best_likely_infection.infector_name}
    return logs + [best_likely_infection]

def get_patient_0(user):
    if user not in infectee_infector or user == PATIENT_0:
        return user
    else:
        return get_patient_0(infectee_infector[user])

def fix_ancestry():
    infectee_data = {}
    missing_infectors = {}
    for _fname in get_files():
        print(_fname)
        df = get_file(_fname)
        df.sort_values('post_dt', inplace=True)

        removal_indexes = []
        for index, row in df.iterrows():
            if row.infectee_name in infectee_data:
                # duplicate infection
                removal_indexes.append(index)
                continue
            if row.infectee_name in missing_infectors:
                # a loop was detected in conjunction with missing data, this gets handled later
                continue
            if type(row.infector_name) is not str and np.isnan(row.infector_name):
                infectee_data[row.infectee_name] = {'post_dt': row.post_dt.replace(tzinfo=pytz.timezone('America/New_York')), 'infector_name': ''}
                continue
            infectee_data[row.infectee_name] = {'post_dt': row.post_dt.replace(tzinfo=pytz.timezone('America/New_York')), 'infector_name': row.infector_name}
            if row.infector_name and row.infector_name not in infectee_data:
                # missing infector infection
                if row.infector_name not in missing_infectors or row.post_dt.replace(tzinfo=pytz.timezone('America/New_York')) < missing_infectors[row.infector_name]:
                    missing_infectors[row.infector_name] = row.post_dt.replace(tzinfo=pytz.timezone('America/New_York'))
        # remove rows and write new log
        if removal_indexes:
            df.sort_index(inplace=True)
            df.drop(removal_indexes, inplace=True)
            df.to_csv(_fname, sep='\t', header=None, index=False)
            print("{} indexes removed".format(len(removal_indexes)))
    return infectee_data, missing_infectors

def get_missing_ancestry(infectee_data, missing_infectors):
    print("Getting missing ancestry for {} infectors".format(len(missing_infectors)))
    infection_groups = []
    failure_groups = []
    for infector_name, post_dt in missing_infectors.items():
        try:
            infection_logs = get_missing_infection_logs(infector_name, None, infectee_data, max_date=post_dt)
        except Exception as e:
            print(e)
            failure_groups.append((e, infector_name))
        else:
            infection_groups.append(infection_logs)
    return infection_groups, failure_groups

def fix_missing_ancestry(infection_groups):
    # add logs to corresponding log files :)
    for infection_logs in infection_groups:
        for log in infection_logs:
            dt = parser.parse(log.post_dt).replace(tzinfo=pytz.timezone('America/New_York'))
            _fname = '{}/memes_infections.{}.{}.log'.format(get_files(), dt.day, dt.hour)
            df = get_file(_fname)
            print("writing new log to file: {} + {}".format(_fname, log))
            df.append(log._asdict(), ignore_index=True)
            df.sort_values('post_dt', inplace=True)
            df.to_csv(_fname, sep='\t', header=None, index=False)

# TODO: remove sub-tree from log files :(

def fix_ancestry_2():
    infectee_data = {}
    invalid_infectors = set()
    invalid_roots = set()
    for _fname in get_files():
        print(_fname)
        df = get_file(_fname)
        df.sort_values('post_dt', inplace=True)

        removal_indexes = []
        for index, row in df.iterrows():
            if row.infectee_name in infectee_data:
                # duplicate infection
#                 print("duplicate: {}".format(row))
                removal_indexes.append(index)
                continue
            if row.infector_name in invalid_infectors:
                # invalid infection
                invalid_infectors.add(row.infectee_name)
#                 print("invalid2: {}".format(row))
#                 removal_indexes.append(index)
                continue
            if type(row.infector_name) is not str and np.isnan(row.infector_name):
                infectee_data[row.infectee_name] = ''
                continue
            if row.infector_name in infectee_data:
                infectee_data[row.infectee_name] = row.infector_name
            else:
                # invalid infection
                invalid_roots.add(row.infector_name)
                invalid_infectors.add(row.infector_name)
                invalid_infectors.add(row.infectee_name)
#                 print("invalid1: {}".format(row))
#                 removal_indexes.append(index)
        # remove rows and write new log
        if removal_indexes:
            df.sort_index(inplace=True)
            df.drop(removal_indexes, inplace=True)
            df.to_csv(_fname, sep='\t', header=None, index=False)
            print("{} indexes removed".format(len(removal_indexes)))
    return infectee_data, invalid_infectors, invalid_roots

In [5]:
infectee_data, missing_infectors = fix_ancestry()

../../data/memes_infections.20.19.log
../../data/memes_infections.20.20.log
../../data/memes_infections.20.21.log
../../data/memes_infections.20.22.log
../../data/memes_infections.20.23.log
../../data/memes_infections.21.0.log
../../data/memes_infections.21.1.log
../../data/memes_infections.21.2.log
../../data/memes_infections.21.3.log
../../data/memes_infections.21.4.log
../../data/memes_infections.21.5.log
../../data/memes_infections.21.6.log
../../data/memes_infections.21.7.log
../../data/memes_infections.21.8.log
../../data/memes_infections.21.9.log
../../data/memes_infections.21.10.log
../../data/memes_infections.21.11.log
../../data/memes_infections.21.12.log
../../data/memes_infections.21.13.log
../../data/memes_infections.21.14.log
../../data/memes_infections.21.15.log
../../data/memes_infections.21.16.log
../../data/memes_infections.21.17.log
../../data/memes_infections.21.18.log
../../data/memes_infections.21.19.log
../../data/memes_infections.21.20.log
../../data/memes_infec

../../data/memes_infections.29.23.log
../../data/memes_infections.30.0.log
../../data/memes_infections.30.1.log
../../data/memes_infections.30.2.log
../../data/memes_infections.30.3.log
../../data/memes_infections.30.4.log
../../data/memes_infections.30.5.log
../../data/memes_infections.30.6.log
../../data/memes_infections.30.7.log
../../data/memes_infections.30.8.log
../../data/memes_infections.30.9.log
../../data/memes_infections.30.10.log
../../data/memes_infections.30.11.log
../../data/memes_infections.30.12.log
../../data/memes_infections.30.13.log
../../data/memes_infections.30.14.log
../../data/memes_infections.30.15.log
../../data/memes_infections.30.16.log
../../data/memes_infections.30.17.log
../../data/memes_infections.30.18.log
../../data/memes_infections.30.19.log
../../data/memes_infections.30.20.log
../../data/memes_infections.30.21.log
../../data/memes_infections.30.22.log
../../data/memes_infections.30.23.log
../../data/memes_infections.31.0.log
../../data/memes_infect

In [203]:
missing_infectors

{'u/5amukai': Timestamp('2020-03-25 13:30:43.952444'),
 'u/A_shovel_': Timestamp('2020-03-21 14:49:48.375981'),
 'u/AlbertXFish': Timestamp('2020-03-25 22:59:59.499948'),
 'u/Anoobis100percent': Timestamp('2020-03-25 04:19:21.330793'),
 'u/AutisthicccGuy': Timestamp('2020-03-28 06:15:29.945040'),
 'u/BeardPhile': Timestamp('2020-03-25 04:47:21.692608'),
 'u/Bit_of_a_Hater': Timestamp('2020-04-02 03:28:20.583997'),
 'u/BoomBug': Timestamp('2020-03-31 13:21:04.757940'),
 'u/C0uchP0t4tO': Timestamp('2020-03-21 18:56:08.135622'),
 'u/Ca-Game': Timestamp('2020-03-25 15:21:45.565364'),
 'u/Chong_Long_Dong': Timestamp('2020-03-25 04:12:23.831106'),
 'u/Cottoncandyskiesthrw': Timestamp('2020-03-21 14:53:00.572599'),
 'u/DieHarD04': Timestamp('2020-03-21 18:08:56.406433'),
 'u/Drodr10': Timestamp('2020-03-21 15:31:52.341864'),
 'u/Durmath': Timestamp('2020-04-02 16:19:42.120531'),
 'u/ETHEKILLER': Timestamp('2020-03-21 18:46:52.815095'),
 'u/EbbieXinYue': Timestamp('2020-03-27 18:02:00.779113')

In [6]:
infection_groups, failure_groups = get_missing_ancestry(infectee_data, missing_infectors)

Getting missing ancestry for 114 infectors
Getting missing infection logs for user 'u/EbbieXinYue' from infectee 'None'
Getting missing infection logs for user 'u/iam_inevtable' from infectee 'None'
Getting missing infection logs for user 'u/jimmydcriket' from infectee 'u/iam_inevtable'
Getting missing infection logs for user 'u/aranzord_sama' from infectee 'u/jimmydcriket'
Getting missing infection logs for user 'u/AlexysYoung' from infectee 'u/jimmydcriket'
Getting missing infection logs for user 'u/kirk_s7' from infectee 'u/AlexysYoung'
Getting missing infection logs for user 'u/_insert_Nick_here_' from infectee 'u/kirk_s7'
Getting missing infection logs for user 'u/Supergamer5465' from infectee 'u/_insert_Nick_here_'
Getting missing infection logs for user 'u/bigboomer420' from infectee 'u/Supergamer5465'
Getting missing infection logs for user 'u/Courtlander1993' from infectee 'u/bigboomer420'
Getting missing infection logs for user 'u/alex_NSI' from infectee 'u/Courtlander1993'
G

Getting missing infection logs for user 'u/PredatorOwl' from infectee 'u/thisnameis_'
Getting missing infection logs for user 'u/KreagerStein' from infectee 'u/PredatorOwl'
Getting missing infection logs for user 'u/magic-soup' from infectee 'u/KreagerStein'
Getting missing infection logs for user 'u/Hungry-Carrot13' from infectee 'u/magic-soup'
Getting missing infection logs for user 'u/AVeryLonelyWeeb' from infectee 'u/Hungry-Carrot13'
Getting missing infection logs for user 'u/Sungami00' from infectee 'u/Hungry-Carrot13'
Getting missing infection logs for user 'u/ServantOfTheLost' from infectee 'u/Sungami00'
Getting missing infection logs for user 'u/DrAhktarious' from infectee 'u/Sungami00'
Getting missing infection logs for user 'u/_Foxified_' from infectee 'u/DrAhktarious'
Getting missing infection logs for user 'u/Rede331' from infectee 'u/DrAhktarious'
Getting missing infection logs for user 'u/StonnerShaggy' from infectee 'u/DrAhktarious'
Getting missing infection logs for use

Getting missing infection logs for user 'u/brydon9' from infectee 'u/88xfi'
Getting missing infection logs for user 'u/rap1dbunny' from infectee 'u/brydon9'
Getting missing infection logs for user 'u/Dill_PickleOG' from infectee 'u/rap1dbunny'
Getting missing infection logs for user 'u/xavier7777777' from infectee 'u/Dill_PickleOG'
Getting missing infection logs for user 'u/Lars82NL' from infectee 'u/xavier7777777'
Getting missing infection logs for user 'u/totalyabove18' from infectee 'u/Lars82NL'
Getting missing infection logs for user 'u/HotHotPotato79' from infectee 'u/Lars82NL'
Getting missing infection logs for user 'u/gloooooooo' from infectee 'u/Lars82NL'
Getting missing infection logs for user 'u/LordRaisinBran' from infectee 'u/gloooooooo'
Getting missing infection logs for user 'u/yeahggiguess' from infectee 'u/LordRaisinBran'
Getting missing infection logs for user 'u/rslashFuryKiller' from infectee 'u/yeahggiguess'
Getting missing infection logs for user 'u/ChampXXXII' fro

Getting missing infection logs for user 'u/ZoqfotWasTaken' from infectee 'u/SippycupStudios'
Getting missing infection logs for user 'u/MattioC' from infectee 'u/toby_ornautobey'
Getting missing infection logs for user 'u/Alex_Cormier' from infectee 'u/MattioC'
Getting missing infection logs for user 'u/dewottoshawott' from infectee 'u/CasualMetalHead'
Getting missing infection logs for user 'u/B10B3RT0' from infectee 'u/CasualMetalHead'
Getting missing infection logs for user 'u/TerraGamer9384' from infectee 'u/freshmaster69'
Getting missing infection logs for user 'u/DaRealXalien' from infectee 'u/TerraGamer9384'
Getting missing infection logs for user 'u/Wezard_the_MemeLord' from infectee 'u/TerraGamer9384'
Getting missing infection logs for user 'u/ThatAsianBoi21' from infectee 'u/Wezard_the_MemeLord'
Getting missing infection logs for user 'u/SHREKYUMTUM69' from infectee 'u/ThatAsianBoi21'
Getting missing infection logs for user 'u/Jax_Destro' from infectee 'u/SHREKYUMTUM69'
Getti

Retrying due to ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='oauth.reddit.com', port=443): Read timed out. (read timeout=16.0)",),) status: GET https://oauth.reddit.com/comments/fokz4j/


...

Retrying due to ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='oauth.reddit.com', port=443): Read timed out. (read timeout=16.0)",),) status: GET https://oauth.reddit.com/comments/dn3mob/


Getting missing infection logs for user 'u/beannie_babbiiee' from infectee 'None'


Retrying due to ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='oauth.reddit.com', port=443): Read timed out. (read timeout=16.0)",),) status: GET https://oauth.reddit.com/user/beannie_babbiiee/comments
Retrying due to ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='oauth.reddit.com', port=443): Read timed out. (read timeout=16.0)",),) status: GET https://oauth.reddit.com/comments/folyfm/


Failed to get comments for user 'u/beannie_babbiiee': received 200 HTTP response
Getting missing infection logs for user 'u/TheSCP_3008' from infectee 'None'


Retrying due to ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='oauth.reddit.com', port=443): Read timed out. (read timeout=16.0)",),) status: GET https://oauth.reddit.com/user/TheSCP_3008/comments


In [ ]:
infection_groups

In [178]:
fix_missing_ancestry(infection_groups)

FileNotFoundError: [Errno 2] File b'../../data/memes_infections.6.22.log' does not exist: b'../../data/memes_infections.6.22.log'

In [139]:
sr = _reddit.subreddit('memes').new(limit=1)
com = list(sr)[0]
com.id

'fv7eoi'

In [204]:
dftest = pandas.concat([get_file(f) for f in all_files])
dftest[dftest['infectee_name'] == 'u/AutoModerator']

,post_dt,infectee_name,infectee_post_id,infector_name,infector_post_id,comment_or_submission
136,2020-03-21 07:54:50.598404,u/AutoModerator,fl3n8gg,u/embracethemems,fmduoi,S


In [12]:
infectee_data_2, invalid_infectors, invalid_roots = fix_ancestry_2()

../../data/memes_infections.20.19.log
../../data/memes_infections.20.20.log
../../data/memes_infections.20.21.log
../../data/memes_infections.20.22.log
../../data/memes_infections.20.23.log
../../data/memes_infections.21.0.log
../../data/memes_infections.21.1.log
../../data/memes_infections.21.2.log
../../data/memes_infections.21.3.log
../../data/memes_infections.21.4.log
../../data/memes_infections.21.5.log
../../data/memes_infections.21.6.log
../../data/memes_infections.21.7.log
../../data/memes_infections.21.8.log
../../data/memes_infections.21.9.log
../../data/memes_infections.21.10.log
../../data/memes_infections.21.11.log
../../data/memes_infections.21.12.log
../../data/memes_infections.21.13.log
../../data/memes_infections.21.14.log
../../data/memes_infections.21.15.log
../../data/memes_infections.21.16.log
../../data/memes_infections.21.17.log
../../data/memes_infections.21.18.log
../../data/memes_infections.21.19.log
../../data/memes_infections.21.20.log
../../data/memes_infec

10 indexes removed
../../data/memes_infections.27.14.log
7 indexes removed
../../data/memes_infections.27.15.log
8 indexes removed
../../data/memes_infections.27.16.log
5 indexes removed
../../data/memes_infections.27.17.log
5 indexes removed
../../data/memes_infections.27.18.log
6 indexes removed
../../data/memes_infections.27.19.log
11 indexes removed
../../data/memes_infections.27.20.log
7 indexes removed
../../data/memes_infections.27.21.log
6 indexes removed
../../data/memes_infections.27.22.log
23 indexes removed
../../data/memes_infections.27.23.log
14 indexes removed
../../data/memes_infections.28.0.log
11 indexes removed
../../data/memes_infections.28.1.log
7 indexes removed
../../data/memes_infections.28.2.log
4 indexes removed
../../data/memes_infections.28.3.log
11 indexes removed
../../data/memes_infections.28.4.log
14 indexes removed
../../data/memes_infections.28.5.log
7 indexes removed
../../data/memes_infections.28.6.log
3 indexes removed
../../data/memes_infections.28

4 indexes removed
../../data/memes_infections.2.18.log
5 indexes removed
../../data/memes_infections.2.19.log
12 indexes removed
../../data/memes_infections.2.20.log
5 indexes removed
../../data/memes_infections.2.21.log
6 indexes removed
../../data/memes_infections.2.22.log
10 indexes removed
../../data/memes_infections.2.23.log
16 indexes removed
../../data/memes_infections.3.0.log
7 indexes removed
../../data/memes_infections.3.1.log
../../data/memes_infections.3.2.log
3 indexes removed
../../data/memes_infections.3.3.log
5 indexes removed
../../data/memes_infections.3.4.log
4 indexes removed
../../data/memes_infections.3.5.log
4 indexes removed
../../data/memes_infections.3.6.log
4 indexes removed
../../data/memes_infections.3.7.log
7 indexes removed
../../data/memes_infections.3.8.log
9 indexes removed
../../data/memes_infections.3.9.log
10 indexes removed
../../data/memes_infections.3.10.log
10 indexes removed
../../data/memes_infections.3.11.log
4 indexes removed
../../data/meme

10 indexes removed
../../data/memes_infections.9.3.log
8 indexes removed
../../data/memes_infections.9.4.log
4 indexes removed
../../data/memes_infections.9.5.log
11 indexes removed
../../data/memes_infections.9.6.log
11 indexes removed
../../data/memes_infections.9.7.log
11 indexes removed
../../data/memes_infections.9.8.log
7 indexes removed
../../data/memes_infections.9.9.log
7 indexes removed
../../data/memes_infections.9.10.log
7 indexes removed
../../data/memes_infections.9.11.log
12 indexes removed
../../data/memes_infections.9.12.log
7 indexes removed
../../data/memes_infections.9.13.log
../../data/memes_infections.9.14.log
2 indexes removed
../../data/memes_infections.9.15.log
4 indexes removed
../../data/memes_infections.9.16.log
5 indexes removed
../../data/memes_infections.9.17.log
2 indexes removed
../../data/memes_infections.9.18.log
10 indexes removed
../../data/memes_infections.9.19.log
7 indexes removed
../../data/memes_infections.9.20.log
8 indexes removed
../../data/

8 indexes removed
../../data/memes_infections.15.7.log
10 indexes removed
../../data/memes_infections.15.8.log
3 indexes removed
../../data/memes_infections.15.9.log
8 indexes removed
../../data/memes_infections.15.10.log
6 indexes removed
../../data/memes_infections.15.11.log
6 indexes removed
../../data/memes_infections.15.12.log
17 indexes removed
../../data/memes_infections.15.13.log
8 indexes removed
../../data/memes_infections.15.14.log
7 indexes removed
../../data/memes_infections.15.15.log
3 indexes removed
../../data/memes_infections.15.16.log
5 indexes removed
../../data/memes_infections.15.17.log
9 indexes removed
../../data/memes_infections.15.18.log
4 indexes removed
../../data/memes_infections.15.19.log
5 indexes removed
../../data/memes_infections.15.20.log
12 indexes removed
../../data/memes_infections.15.21.log
8 indexes removed
../../data/memes_infections.15.22.log
3 indexes removed
../../data/memes_infections.15.23.log
12 indexes removed
../../data/memes_infections.1

In [219]:
import networkx as nx

infection_graph = nx.DiGraph()

nodes = set(infectee_data_2) | set(infectee_data_2.values())
infection_graph.add_nodes_from(nodes)
infection_graph.add_edges_from([(v, k) for k,v in infectee_data_2.items()])
print(nx.algorithms.dag.is_directed_acyclic_graph(infection_graph))
top_sort = iter(nx.algorithms.dag.topological_sort(infection_graph))
for _ in range(3):
    print(next(top_sort))

True

u/woodendoors7
u/personface15
